# テキストデータの可視化

[青空文庫](https://www.aozora.gr.jp/)に収録されている、夏目漱石の[『三四郎』](https://www.aozora.gr.jp/cards/000148/card794.html)という作品を使って、テキストデータの可視化を練習します。

自然言語処理のフレームワークである[spaCy](https://spacy.io/)とこれを利用した日本語NLPライブラリである[GiNZA](https://megagonlabs.github.io/ginza/)を使います。

![](https://spacy.io/pipeline-fde48da9b43661abcdf62ab70a546d71.svg)

spaCyでは、各コンポーネント（機能）が順番に適用されるpipeline方式でテキストが処理されます。基本的には、tokenizer（分かち書き）、tagger（品詞付与）、parser（係り受け解析）、ner（固有表現抽出）、lemmatizer（原形抽出）、textcat（文書分類）というコンポーネントが用意されています。

`nlp()`を実行するとデフォルトでtokenizer、tagger、parser、ner、lemmatizerが入力文書に適用されます。

spaCyとGiNZAと比べて、より長く使われてきたライブラリとして、

* [NLTK (Natural Language Toolkit)](https://www.nltk.org/index.html)
* [MeCab](https://taku910.github.io/mecab/)（形態素解析） + [CaboCha](https://taku910.github.io/cabocha/)（係り受け解析）
* [JUMAN](https://nlp.ist.i.kyoto-u.ac.jp/?JUMAN)（形態素解析） + [KNP](https://nlp.ist.i.kyoto-u.ac.jp/?KNP)（係り受け解析）

などがあります。

より高度な分析をする時に必要となることがあるので、ぜひチェックしてみてください。

それでは、spaCyを使った形態素解析の方法を確認してから、テキストデータの可視化を行いましょう。

## 形態素解析の練習

形態素解析は、文章を一つ一つの形態素に分ける技術です。形態素は、「言葉が意味を持つまとまりの単語の最小単位」です。

In [1]:
# ライブラリのインストール（初回のみ）

!pip install spacy ginza ja-ginza
!pip install scikit-learn pandas

     ---------------------------------------- 12.2/12.2 MB 8.6 MB/s eta 0:00:00
  Using cached ginza-5.2.0-py3-none-any.whl (21 kB)
  Using cached ja_ginza-5.2.0-py3-none-any.whl (59.1 MB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl (29 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl (22 kB)
  Using cached weasel-0.4.1-py3-none-any.whl (50 kB)
  Using cached typer-0.15.1-py3-none-any.whl (44 kB)
  Using cached langcodes-3.5.0-py3-none-any.whl (182 kB)
     ---------------------------------------- 1.5/1.5 MB 10.5 MB/s eta 0:00:00
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl (133 kB)
     -------------------------------------- 122.2/122.2 KB 7.5 MB/s eta 0:00:00
     ------------------------------------- 431.8/431.8 KB 13.6 MB/s eta 0:00:00
  Using cached wasabi-1.1.3-py3-none-any.whl (27 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl (17 kB)
     ------------------------------------- 632.3/632.3 KB 13.2 MB

You should consider upgrading via the 'C:\Users\tanri\infovis\infovis-notebooks\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\tanri\infovis\infovis-notebooks\.venv\Scripts\python.exe -m pip install --upgrade pip' command.



     ---------------------------------------- 11.1/11.1 MB 8.4 MB/s eta 0:00:00
     ---------------------------------------- 11.6/11.6 MB 7.5 MB/s eta 0:00:00
  Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
     ---------------------------------------- 44.8/44.8 MB 7.4 MB/s eta 0:00:00
  Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)


### 日本語

spaCyで日本語の形態素解析モデル（`ja_ginza`）をロードして、形態素解析をしてみましょう。

In [2]:
import spacy

nlp = spacy.load("ja_ginza")
text = "これは文章です。"
doc = nlp(text)
for token in doc:
    print(token)

これ
は
文章
です
。


形態素解析の結果には、語の原形や品詞の情報も含まれます。

In [3]:
text = "うとうととして目がさめると女はいつのまにか、隣のじいさんと話を始めている。このじいさんはたしかに前の前の駅から乗ったいなか者である。発車まぎわに頓狂な声を出して駆け込んで来て、いきなり肌をぬいだと思ったら背中にお灸のあとがいっぱいあったので、三四郎の記憶に残っている。じいさんが汗をふいて、肌を入れて、女の隣に腰をかけたまでよく注意して見ていたくらいである。"
doc = nlp(text)
for token in doc:
    print(f"{token}\t{token.lemma_}\t{token.pos_}\t{token.tag_}")

うとうと	うとうと	ADV	副詞
と	と	ADP	助詞-格助詞
し	する	AUX	動詞-非自立可能
て	て	SCONJ	助詞-接続助詞
目	目	NOUN	名詞-普通名詞-一般
が	が	ADP	助詞-格助詞
さめる	さめる	VERB	動詞-一般
と	と	SCONJ	助詞-格助詞
女	女	NOUN	名詞-普通名詞-一般
は	は	ADP	助詞-係助詞
いつ	いつ	PRON	代名詞
の	の	ADP	助詞-格助詞
ま	ま	NOUN	名詞-普通名詞-助数詞可能
に	に	ADP	助詞-格助詞
か	か	ADP	助詞-副助詞
、	、	PUNCT	補助記号-読点
隣	隣	NOUN	名詞-普通名詞-一般
の	の	ADP	助詞-格助詞
じい	じい	NOUN	名詞-普通名詞-一般
さん	さん	NOUN	接尾辞-名詞的-一般
と	と	ADP	助詞-格助詞
話	話	NOUN	名詞-普通名詞-サ変可能
を	を	ADP	助詞-格助詞
始め	始める	VERB	動詞-非自立可能
て	て	SCONJ	助詞-接続助詞
いる	いる	VERB	動詞-非自立可能
。	。	PUNCT	補助記号-句点
この	この	DET	連体詞
じい	じい	NOUN	名詞-普通名詞-一般
さん	さん	NOUN	接尾辞-名詞的-一般
は	は	ADP	助詞-係助詞
たしか	たしか	ADJ	形状詞-一般
に	だ	AUX	助動詞
前	前	NOUN	名詞-普通名詞-副詞可能
の	の	ADP	助詞-格助詞
前	前	NOUN	名詞-普通名詞-副詞可能
の	の	ADP	助詞-格助詞
駅	駅	NOUN	名詞-普通名詞-一般
から	から	ADP	助詞-格助詞
乗っ	乗る	VERB	動詞-一般
た	た	AUX	助動詞
いなか者	いなか者	PROPN	名詞-普通名詞-一般
で	だ	AUX	助動詞
ある	ある	VERB	動詞-非自立可能
。	。	PUNCT	補助記号-句点
発車	発車	NOUN	名詞-普通名詞-サ変可能
まぎわ	まぎわ	NOUN	名詞-普通名詞-一般
に	に	ADP	助詞-格助詞
頓狂	頓狂	VERB	形状詞-一般
な	だ	AUX	助動詞
声	声	NOUN	名詞-普通名詞-一般
を	を	ADP	助詞-格助詞
出し	出す	VERB	動詞-非自立可能
て	て	SCONJ	助詞-接続助詞
駆け込ん	駆け込む	VERB	動詞-一般
で	で	SCONJ

### 英語

spaCyでは、モデルを変えるだけで同じ手順で他言語の解析を行うことができます。

英語のモデルでも試してみましょう。

In [4]:
# 英語のモデルをダウンロードする
!python -m spacy download en_core_web_sm

     ---------------------------------------- 12.8/12.8 MB 9.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


You should consider upgrading via the 'c:\Users\tanri\infovis\infovis-notebooks\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [5]:
nlp = spacy.load('en_core_web_sm')
text = "This is a sentence."
doc = nlp(text)
for token in doc:
    print(f"{token}\t{token.lemma_}\t{token.pos_}\t{token.tag_}")

This	this	PRON	DT
is	be	AUX	VBZ
a	a	DET	DT
sentence	sentence	NOUN	NN
.	.	PUNCT	.


## テキストデータの可視化

それでは、『三四郎』を題材にテキストデータの可視化を行ってみましょう。

* ワードクラウド
* 共起ネットワーク
* 共起マトリックス

の3種類を紹介します。

### データの用意

まずは、青空文庫からテキストデータをダウンロードします。解凍されたファイルの文字コードがShift-JISになっている点に注意しましょう。

In [10]:
# ファイルをダウンロードする
!curl https://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip -o 794_ruby_4237.zip
# textフォルダ作る
!mkdir text
# ファイルをtextフォルダに解凍
# !unzip -d text -o 794_ruby_4237.zip
!tar -xf 794_ruby_4237.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  154k  100  154k    0     0   448k      0 --:--:-- --:--:-- --:--:--  455k
A subdirectory or file text already exists.


次に、正規表現で青空文庫のルビ、注、アクセントの記号を取り除きます。

同時に、文字コードをShift-JISからUTF-8にします。

```

-------------------------------------------------------
【テキスト中に現れる記号について】

《》：ルビ
（例）頓狂《とんきょう》

｜：ルビの付く文字列の始まりを特定する記号
（例）福岡県｜京都郡《みやこぐん》

［＃］：入力者注　主に外字の説明や、傍点の位置の指定
　　　（数字は、JIS X 0213の面区点番号またはUnicode、底本のページと行数）
（例）※［＃「魚＋師のつくり」、第4水準2-93-37］

〔〕：アクセント分解された欧文をかこむ
（例）〔ve'rite'《ヴェリテ》 vraie《ヴレイ》.〕
アクセント分解についての詳細は下記URLを参照してください
http://www.aozora.gr.jp/accent_separation.html
-------------------------------------------------------

```

In [6]:
import re

input_fn = "text/sanshiro.txt"
output_fn = "text/sanshiro.stripruby.txt"

with open(input_fn, encoding="shift_jis") as fin, open(output_fn, mode="w") as fout:
    for line in fin:
        fout.write(re.sub(r"《[^》]+》|［[^］]+］|〔[^〕]+〕| [｜]", "", line))

冒頭と末尾の説明を取り除きます（何行取り除くかは目視で確認）。

In [2]:
import sys

# macOSの場合は、homebrewでcoreutilsをインストールする必要があります（gheadを使うため）
# homebrewがインストールされていない場合は、https://brew.sh/からインストールしてください

if sys.platform == "darwin":
    !brew install coreutils
    !tail -n +22 text/sanshiro.stripruby.txt | ghead -n -14 > text/sanshiro.corpus.txt
else:
    !tail -n +22 text/sanshiro.stripruby.txt | head -n -14 > text/sanshiro.corpus.txt

'tail' is not recognized as an internal or external command,
operable program or batch file.


これで、テキストファイルを扱う準備ができました。

### ワードクラウド

ワードクラウドは、形態素解析で得られた頻出単語の頻出度合いを文字の大きさで可視化する手法です。頻度の高い単語を大きく表示することで、テキスト全体の傾向を素早く理解することができます。

#### 形態素解析

それでは、『三四郎』に出現する単語の頻度を数えてみましょう。

テキストファイルを読み込んで形態素解析を行います。

In [2]:
import spacy
input_fn = "text/sanshiro.corpus.txt"
output_fn = "text/sanshiro.wakati.txt"

nlp = spacy.load("ja_ginza")


In [3]:
lines = []
with open(input_fn, "r", encoding="shift-jis") as fin:
    for line in fin:
        lines.append(line)

In [4]:
with open(output_fn, "a", encoding="shift-jis") as fout:
    for line in lines:
        tokens = [token.text for token in nlp(line.rstrip())]
    
        fout.write(' '.join(tokens) + "\n")

: 

出力されたファイルを確認すると、単語分割されていることが分かります。

次に、使用頻度の高い単語を見てみましょう。

In [3]:
# ファイルを読み込み、テキストを一行ずつ解析
all_tokens = []
with open(input_fn, "r", encoding="utf-8") as f:
    for line in f:
        tokens = [token for token in nlp(line)]
        all_tokens.extend(tokens)

: 

In [1]:
from collections import Counter

# 分析対象とする品詞（内容語 - 名詞、動詞、形容詞）と不要語（ストップワード）を指定する
include_pos = ("NOUN", "VERB", "ADJ")
stopwords = ("する", "ある", "ない", "いう", "もの", "こと", "よう", "なる", "ほう", "いる", "くる", "さん")

# 単語の頻度を数える
counter = Counter(token.lemma_ for token in all_tokens if token.pos_ in include_pos and token.lemma_ not in stopwords)

# 出現頻度top 20を出力する
for word, count in counter.most_common(20):
    print(f"{count:>5} {word}")

NameError: name 'all_tokens' is not defined

#### ワードクラウドの作成

それでは、このデータをもとにワードクラウドを作成します。

In [ ]:
# ライブラリをインストールする
!pip install wordcloud

In [ ]:
# 日本語フォントのダウンロード（Linuxのみ）

if sys.platform == "linux":
    !sudo apt update
    !sudo apt install fonts-ipaexfont

In [ ]:
# フォントファイルの場所の指定（図が上手く表示されない場合は、書き換えてください）

if sys.platform == "darwin":
    fpath = "/Library/Fonts/Arial Unicode.ttf"
else:
    fpath = "/usr/share/fonts/opentype/ipaexfont-gothic/ipaexg.ttf"

In [ ]:
words = [token.lemma_ for token in all_tokens if token.pos_ in include_pos and token.lemma_ not in stopwords]

In [ ]:
words

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# ワードクラウドの表示設定と作成
wordcloud = WordCloud(
    width=1600, height=800,
    background_color="white", font_path=fpath
).generate(' '.join(words))

# プロット
plt.figure(figsize=(8, 4))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.tight_layout(pad=0)
plt.savefig("wordcloud.png", dpi=300)
plt.show()

「先生」、「言う」、「女」、「見る」などが高頻度で出現していることが分かります。

分析対象の品詞に、固有名詞を加えてみたらどうでしょうか？試してみましょう。

In [ ]:
# 分析対象とする品詞と不要語を指定する
# your code goes in ????? below

include_pos = ("NOUN", "VERB", "ADJ", "?????")
stopwords = ("する", "ある", "ない", "いう", "もの", "こと", "よう", "なる", "ほう", "いる", "くる")

In [ ]:
words = [token.lemma_ for token in all_tokens if token.pos_ in include_pos and token.lemma_ not in stopwords]

In [ ]:
# ワードクラウドの表示設定と作成
wordcloud = WordCloud(
    width=1600, height=800,
    background_color="white", font_path=fpath
).generate(' '.join(words))

# プロット
plt.figure(figsize=(8, 4))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

### 共起ネットワーク

次に、共起ネットワークを作って、どの語とどの語が一緒に使われているかを調べてみましょう。

#### 共起分析

まず、共起分析を行います。文章を文に分割し、同一文中に同時に出現する単語の組を数え上げることで分析します。

In [3]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
def extract_words(sent, pos_tags, stopwords):
    """
    分析対象の品詞であり、不要語ではない単語を抽出する
    """
    words = [token.lemma_ for token in sent if token.pos_ in pos_tags and token.lemma_ not in stopwords]
    return words

In [ ]:
def count_cooccurrence(sents, token_length="{2,}"):
    """
    同じ文中に共起する単語を行列形式で列挙する
    """
    token_pattern = f"\\b\\w{token_length}\\b"
    count_model = CountVectorizer(token_pattern=token_pattern)

    X = count_model.fit_transform(sents)
    words = count_model.get_feature_names_out()
    word_counts = np.asarray(X.sum(axis=0)).reshape(-1)

    X[X > 0] = 1 # 同じ共起が2以上出現しても1とする
    Xc = (X.T * X) # 共起行列を求めるための掛け算をする、csr形式の疎行列

    return words, word_counts, Xc, X

In [2]:
def find_sentence_by_cooccurrence(X, idxs):
    """
    指定された共起を含む文を見つける
    """
    occur_flags = (X[:, idxs[0]] > 0)
    for idx in idxs[1:]:
        occur_flags = occur_flags.multiply(X[:, idx] > 0)

    return occur_flags.nonzero()[0]

In [ ]:
# 文章を解析し、共起を求める
include_pos = ("NOUN", "VERB", "ADJ", "PROPN")
stopwords = ("する", "ある", "ない", "いう", "もの", "こと", "よう", "なる", "ほう", "いる", "くる", "さん")

sents = []
with open(input_fn, "r") as f:
    for line in f:
        doc = nlp(line)
        tmp = [' '.join(extract_words(sent, include_pos, stopwords)) for sent in doc.sents]
        sents.extend(tmp)

words, word_counts, Xc, X = count_cooccurrence(sents)

In [ ]:
# 共起ランキングを出力する
# 共起行列Xcは疎行列なので、非ゼロ要素のみをカウンタに格納する
counter = Counter()
for i, j in zip(*Xc.nonzero()):
    if i >= j:
        continue
    counter[(i, j)] += Xc[i, j]

# 共起の出現頻度top 20を出力する
for (i, j), c in counter.most_common(20):
    print(f"{c:>3d} ({words[i]}, {words[j]})")

In [ ]:
# 指定した共起を含む文のリストを出力する
sents_orig = []
with open(input_fn, "r") as f:
    for line in f:
        doc = nlp(line)
        tmp = list(doc.sents)
        sents_orig.extend(tmp)

# すべての単語の通し番号を求める
words_lookup = { word: index for index, word in enumerate(words) }

In [ ]:
# 共起語を指定する
lookup_words = ["野々宮", "美禰子"]

# 指定した共起語のインデックスを求める
idxs = list(map(lambda x: words_lookup[x], lookup_words))

# 指定した共起を含む文のリストを出力する
for i in find_sentence_by_cooccurrence(X, idxs):
    print(f"{i:>5d}: {sents_orig[i]}")

#### 共起ネットワークの作成

共起分析の結果に基づいて共起ネットワークを作成します。

In [ ]:
!pip install networkx pyvis japanize_matplotlib

In [ ]:
import networkx as nx
from pyvis.network import Network
import japanize_matplotlib

In [ ]:
def word_weights(words, word_counts):
    """
    単語の最多頻度が1となるような相対値として単語の重みを求める
    """
    count_max = word_counts.max()
    weights = [(word, {"weight": count / count_max})
               for word, count in zip(words, word_counts)]
    return weights

In [ ]:
def cooccurrence_weights(words, Xc, weight_cutoff):
    """
    共起の最多頻度が1となるような相対値として共起の重みを求める
    共起の重みがweight_cutoffより低い共起は除外する
    """
    Xc_max = Xc.max()
    cutoff = weight_cutoff * Xc_max
    weights = [(words[i], words[j], Xc[i, j] / Xc_max)
               for i, j in zip(*Xc.nonzero()) if i < j and Xc[i, j] > cutoff]
    return weights

In [ ]:
def create_network(words, word_counts, Xc, weight_cutoff):
    """
    語、単語頻度、共起行列から共起ネットワークをNetworkX形式で得る
    """
    G = nx.Graph()

    weights_w = word_weights(words, word_counts)
    G.add_nodes_from(weights_w)

    weights_c = cooccurrence_weights(words, Xc, weight_cutoff)
    G.add_weighted_edges_from(weights_c)

    G.remove_nodes_from(list(nx.isolates(G)))
    return G

In [ ]:
def pyplot_network(G):
    """
    NetworkX形式で与えられた共起ネットワークをpyplotで描画する
    """
    plt.figure(figsize=(10, 10))
    pos = nx.spring_layout(G, k=0.1)

    weights_n = np.array(list(nx.get_node_attributes(G, "weight").values()))
    nx.draw_networkx_nodes(G, pos, node_size=300 * weights_n)
    weights_e = np.array(list(nx.get_edge_attributes(G, "weight").values()))
    nx.draw_networkx_edges(G, pos, width=20 * weights_e)

    nx.draw_networkx_labels(G, pos, font_family="IPAexGothic")

    plt.axis("off")
    plt.tight_layout()
    plt.show()

In [ ]:
def nx2pyvis_G(G):
    """
    NetworkX形式で与えられた共起ネットワークをpyvisで描画する
    """
    pyvis_G = Network(width="800px", height="800px", notebook=True)
    for node, attrs in G.nodes(data=True):
        pyvis_G.add_node(node, title=node, size=30 * attrs["weight"])
    for node1, node2, attrs in G.edges(data=True):
        pyvis_G.add_edge(node1, node2, width=20 * attrs["weight"])

    return pyvis_G

In [ ]:
# ネットワークを作る
G = create_network(words, word_counts, Xc, 0.01)

In [ ]:
# 静的ビジュアライゼーション
pyplot_network(G)

In [ ]:
# インタラクティブなビジュアライゼーション
pyvis_G = nx2pyvis_G(G)
pyvis_G.show_buttons()
pyvis_G.show("network.html")

このファイルを同じディレクトリに保存された`network.html`をブラウザで開き、インタラクティブな共起ネットワークを確認してみてください。

### 共起ヒートマップ

共起関係をネットワークではなく、ヒートマップとして表現することもできます。

登場人物の共起ヒートマップを作ってみましょう。

#### データフレームを作る

In [ ]:
# 文章を解析し、共起を求める
include_pos = ("NOUN", "PROPN")
stopwords = ("する", "ある", "ない", "いう", "もの", "こと", "よう", "なる", "ほう", "いる", "くる", "さん")

sents = []
with open(input_fn, "r") as f:
    for line in f:
        doc = nlp(line)
        tmp = [' '.join(extract_words(sent, include_pos, stopwords)) for sent in doc.sents]
        sents.extend(tmp)

words, word_counts, Xc, X = count_cooccurrence(sents)

In [ ]:
# 共起行列Xcは疎行列なので、非ゼロ要素のみをカウンタに格納する
counter = Counter()
for i, j in zip(*Xc.nonzero()):
    if i >= j:
        continue
    counter[(i, j)] += Xc[i, j]

# 共起の出現頻度top 20を出力する
for (i, j), c in counter.most_common(20):
    print(f"{c:>3d} ({words[i]}, {words[j]})")

In [ ]:
# 行名を用意する

columns = set(Xc.nonzero()[0])
columns_text = [words[i] for i in columns]

In [ ]:
# すべて0のデータフレームを用意する

import pandas as pd

df = pd.DataFrame(np.zeros((len(columns), len(columns))), index=columns_text, columns=columns_text)

In [ ]:
# データフレームに頻度を入れる

for cord, count in counter.items():
    df.iloc[cord] = count

In [ ]:
# 登場人物名のみを取り出す

characters = ["三四郎", "広田", "野々宮", "佐々木", "与次郎", "美禰子", "先生", "原口", "里見"]
df_characters = df[characters].filter(items=characters, axis=0)

#### 共起ヒートマップの作成

In [ ]:
!pip install plotly kaleido

In [ ]:
import plotly.express as px

fig = px.imshow(df_characters, color_continuous_scale=px.colors.sequential.Oranges,
                title="『三四郎』の登場人物", width=800, height=800)
fig.update_layout(font=dict(size=16))
fig.show()
fig.write_image("heatmap.png", scale=2)

人々のインタラクションの多寡が示されました。

----

これらの可視化手法は文学作品だけではなく、他の分野のテキストデータにも有効です。

文学作品に限っていうと、複数の作家の作品群を比較すると面白い結果が出るかもしれません。